In [5]:
! python --version

Python 3.7.15


In [3]:
from __future__ import print_function
import os
import matplotlib.pyplot as plt
import seaborn as sn
import cv2
from random import randint
import numpy as np
import pandas as pd
import shutil
import sys

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from PIL import Image

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c 'concetto22'
! mkdir data

In [6]:
from google.colab import files

In [ ]:
! unzip concetto22 -d chal

In [ ]:
!rm '/content/chal/concetto_CDT/concetto_CDT/README_NHATS_Round_10_SP_ClockDrawings.pdf'

In [ ]:
import cv2, os
base_path = "/content/chal/concetto_CDT/concetto_CDT/"
new_path = "/content/fimg/"
if not os.path.exists (new_path) :
        os.mkdir(new_path)
for infile in os.listdir(base_path):
    print ("file : " + infile)
    read = cv2.imread(base_path + infile)
    outfile = infile.split('.')[0] + '.jpg'
    cv2.imwrite(new_path+outfile,read,[int(cv2.IMWRITE_JPEG_QUALITY), 200])

In [ ]:
import pandas as pd
labels = pd.read_csv(r'/content/chal/train.csv')

train_dir =r'/content/fimg'
DR = r"/content/final/"
if not os.path.exists(DR):
    os.mkdir(DR)

for i in range (1,2540):
    # Create subdirectory with `class_name`
    if not os.path.exists (DR + str(labels.iloc[i,1])) :
        os.mkdir(DR + str(labels.iloc[i,1]))
    src_path = train_dir + '/'+ str(labels.iloc[i,0])
    src_path = src_path[:-2]
    src_path = src_path + '.jpg'
    dst_path = DR + str(labels.iloc[i,1])
    try:
        shutil.copy(src_path, dst_path)
        print("sucessful")
    except IOError as e:
        print('Unable to copy file {} to {}'
              .format(src_path, dst_path))
    except:
        print('When try copy file {} to {}, unexpected error: {}'
              .format(src_path, dst_path, sys.exc_info()))

In [9]:
dir = '/content/final/'
import pathlib
dir = pathlib.Path(dir)
image_count = len(list(dir.glob('*/*.jpg')))
print(image_count)
print(list(dir.glob('./*')))

2539
[PosixPath('/content/final/5.0'), PosixPath('/content/final/3.0'), PosixPath('/content/final/1.0'), PosixPath('/content/final/0.0'), PosixPath('/content/final/2.0'), PosixPath('/content/final/4.0')]


In [1]:
batch_size = 32
img_height = 1024
img_width = 1024
epo = 10
sed = 111

In [8]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  dir,
  validation_split=0.05,
  subset="training",
  seed=sed,
  color_mode='grayscale',
  image_size=(img_height, img_width),
  batch_size= batch_size)

TypeError: ignored

In [28]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  dir,
  validation_split=0.05,
  subset="validation",
  seed=sed,
  color_mode='grayscale',
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 2539 files belonging to 6 classes.
Using 126 files for validation.


In [22]:
class_names = train_ds.class_names
print(class_names)

['0.0', '1.0', '2.0', '3.0', '4.0', '5.0']


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [29]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 1024, 1024, 1)
(32,)


In [30]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [31]:
normalization_layer = layers.Rescaling(1./255)

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

In [ ]:
num_classes = len(class_names)

model = Sequential()

from keras import layers
resnet = tf.keras.applications.VGG19(
    include_top=False,
    weights="imagenet",
    input_shape=(img_height,img_width,3),
    pooling='max',
    classifier_activation="softmax",
    classes=6,
)
resnet.trainable = False

model.add(resnet)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.00001)
model.compile(optimizer=opt,              
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=15
)

In [ ]:
import heapq
img = tf.keras.utils.load_img(
    '/content/final/1.0/10004622.jpg', target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])
a=class_names[np.argmax(score)]
b=100 * np.max(score)
k=3
if b<45:
  c = heapq.nlargest(k, range(len(score)), key=score.__getitem__)[k-1]
  a=  class_names[c]
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(a,b )
)

In [ ]:
tf.keras.models.save_model(
    model,
    filepath='/best.h5',
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None,
    save_traces=True
)

In [ ]:
lab = pd.read_csv(r'/test.csv')

for i in range(0,558):
    mad = str(lab.iloc[i,0])
    mad = mad[:-2]
    img = tf.keras.utils.load_img(
    '/content/fimg/'+mad+'.jpg', target_size=(img_height, img_width)
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
    )

In [ ]:
import csv
myList = []
header=['id','tar']
lab = pd.read_csv(r'/test.csv')

for i in range(0,558):
    mad = str(lab.iloc[i,0])
    sad = mad
    mad = mad[:-2]
    img = tf.keras.utils.load_img(
    '/content/fimg/'+mad+'.jpg', target_size=(img_height, img_width)
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    mada=class_names[np.argmax(score)]
    mada=mada[:-2]
    row = [str(sad),str(mada)]
    myList.append(row)


with open('/test.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(myList)
    


In [ ]:
    with open('/test.csv', 'w', encoding='UTF8', newline='') as f:
      writer = csv.writer(f)
    # write the header
      writer.writerow(header)

    # write multiple rows
      writer.writerows(myList)
    